In [3]:
import tqdm

In [4]:
from nnsight import LanguageModel

llm = LanguageModel("openai-community/gpt2", device_map="auto")

print(llm)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): Generator(
    (streamer): Streamer()
  )
)


In [5]:
import torch.nn.functional as F
from scipy.stats import entropy

# Create an entropy calculation function
def calc_entropy(probs, axis=1):
    return entropy(probs, axis=axis)

In [6]:
import nnsight

with llm.trace("The binary representation of the decimal number 170 is "):

    # Get the logits
    logits = llm.lm_head.output.save()

    # Convert logits to probs using softmax
    probs = F.softmax(logits, dim=-1)

    # Convert to token
    pred_token = logits.argmax(dim=-1).save()

    # Calculate entropy on next token probs
    pred_entropy = nnsight.apply(calc_entropy, probs.detach().cpu()).save()

print("Logits:", logits)
print("Entropy:", pred_entropy[0][-1])
print("Token:", llm.tokenizer.decode(pred_token[0][-1]))



Logits: tensor([[[ -36.2874,  -35.0114,  -38.0794,  ...,  -40.5164,  -41.3760,
           -34.9193],
         [ -80.1033,  -79.4853,  -83.6792,  ...,  -85.5055,  -84.5914,
           -82.7390],
         [ -82.4965,  -82.5529,  -85.9854,  ...,  -91.6856,  -89.3994,
           -83.7149],
         ...,
         [-110.9339, -110.5447, -111.2388,  ..., -120.0823, -117.0155,
          -110.3783],
         [-127.2238, -128.5333, -133.2365,  ..., -136.7369, -132.4776,
          -126.5492],
         [ -81.8050,  -83.5660,  -84.7266,  ...,  -92.0885,  -88.4962,
           -84.5517]]], device='mps:0', grad_fn=<LinearBackward0>)
Entropy: 1.2025204
Token:  


In [8]:
import torch
import torch.nn.functional as F
import nnsight
from nnsight import LanguageModel
from scipy.stats import entropy

# Load a language model
llm = LanguageModel("openai-community/gpt2", device_map="auto")

# Function to calculate entropy from probabilities
def calc_entropy(probs, axis=1):
    return entropy(probs, axis=axis)

# Prompt for generation
prompt = "Once upon a time"

# End of sentence tokens (period, question mark, exclamation point)
end_tokens = [13, 30, 0, 986]  # . ? ! ...

In [10]:
with llm.generate(prompt, max_new_tokens=100) as tracer:

    # List to store entropy values and generated tokens
    entropy_values = nnsight.list().save()
    generated_tokens = nnsight.list().save()

    entropy_values.append(1)

print(entropy_values)
print(generated_tokens)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[1]
[]


In [13]:
n_new_tokens = 100

# List to store entropy values and generated tokens
entropy_values = []
generated_tokens = []

# Generate text until a complete sentence is formed
with llm.generate(prompt, max_new_tokens=n_new_tokens) as tracer:

    with llm.lm_head.all():

        # Save the logits
        logits = llm.lm_head.output
        # Convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # Get predicted token
        pred_embed = logits.argmax(dim=-1)
        # Decode token
        pred_token = llm.tokenizer.decode(pred_embed).save()
        # Calculate entropy
        pred_entropy = nnsight.apply(calc_entropy, probs.detach().cpu()).save()

        # Append values to our lists
        entropy_values.append(pred_entropy)
        generated_tokens.append(pred_token)

        if pred_embed in end_tokens:
            break

print(generated_tokens)
print(entropy_values)

AttributeError: 'NoneType' object has no attribute 'end_lineno'

In [68]:
# Old approach:
prompt = 'Once upon a time, '
layers = llm.transformer.h
n_new_tokens = 30
prob_values = []
generated_tokens = []
end_tokens = [243423]
with llm.generate(prompt, max_new_tokens=n_new_tokens) as tracer:

    # First token
    logits = llm.lm_head.output
    probs = F.softmax(logits, dim=-1).save()
    pred_token = logits.argmax(dim=-1)[0][-1].save()
    prob_values.append(probs)
    generated_tokens.append(pred_token)

    # Subsequent tokens
    for i in range(n_new_tokens-1):

        # Save the logits
        logits = llm.lm_head.next().output

        # Convert to probabilities
        probs = F.softmax(logits, dim=-1).save()

        # Get predicted token
        pred_token = logits.argmax(dim=-1)[0][-1].save()

        # Append
        prob_values.append(probs)
        generated_tokens.append(pred_token)

        with tracer.cond(pred_token.item() == 13):
            break


print(generated_tokens)
print(len(generated_tokens))
print(prob_values)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[tensor(1849, device='mps:0'), tensor(40, device='mps:0')]
2
[tensor([[[8.9157e-04, 1.0424e-03, 2.1106e-05,  ..., 7.1022e-07,
          4.8419e-06, 5.4752e-04],
         [3.2892e-06, 1.9758e-05, 3.3276e-07,  ..., 3.1810e-09,
          7.2668e-07, 3.3427e-06],
         [4.0888e-07, 1.1701e-05, 1.7799e-07,  ..., 2.2446e-09,
          1.6359e-06, 1.6848e-05],
         [2.5378e-04, 1.3109e-04, 2.7051e-07,  ..., 2.2008e-07,
          1.6496e-06, 2.9005e-05],
         [8.0199e-06, 2.7697e-06, 2.6061e-07,  ..., 1.2774e-06,
          1.4392e-06, 3.9997e-05],
         [3.3102e-05, 6.4728e-05, 8.0429e-06,  ..., 1.4446e-08,
          8.3535e-07, 2.2285e-06]]], device='mps:0'), tensor([[[1.1516e-05, 4.9644e-03, 1.3941e-04,  ..., 2.8901e-09,
          1.7376e-09, 6.4813e-07]]], device='mps:0')]


In [45]:
for i, token in enumerate(generated_tokens):
    
    tok = token.item()

    print(tok)

    dec_tok = llm.tokenizer.decode(tok)

    print(dec_tok)

1849
 
40
I
373
 was
257
 a
1310
 little
1643
 bit
286
 of
257
 a
4336
 fan
286
 of
262
 the
2656
 original
2168
 series
11
,
475
 but
314
 I
373
 was
635
 also
257
 a
1310
 little
1643
 bit
286
 of
257
 a
4336
 fan
286
 of
262
 the
2656
 original
2168
 series
13
.
314
 I


In [ ]:
        # Save the logits
        logits = llm.lm_head.output
        # Convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # Get predicted token
        pred_embed = logits.argmax(dim=-1)
        # Decode token
        pred_token = llm.tokenizer.decode(pred_embed).save()
        # Calculate entropy
        pred_entropy = nnsight.apply(calc_entropy, probs.detach().cpu()).save()

        # Append values to our lists
        entropy_values.append(pred_entropy)
        generated_tokens.append(pred_token)

        if pred_embed in end_tokens:
            break

In [58]:
prompt = 'Once upon a time, '
n_new_tokens = 30
prob_values = []
generated_tokens = []
end_tokens = [13, 30, 0, 986]  # . ? ! ...

with llm.generate(prompt, max_new_tokens=n_new_tokens) as tracer:
    # Access the first token (works reliably)
    with llm.lm_head.all():
        # Process first token
        logits = llm.lm_head.output
        probs = F.softmax(logits, dim=-1).save()
        pred_token = logits.argmax(dim=-1)[0][-1].save()
        
        # Save values
        prob_values.append(probs)
        generated_tokens.append(pred_token)
    
    # Use a while loop with careful token checks
    token_index = 0
    current_token = pred_token
    
    # Process subsequent tokens
    while token_index < n_new_tokens - 1:
        if current_token.item() in end_tokens:
            # Log the stop condition
            tracer.log(f"Stopping at position {token_index}, token: {current_token.item()}")
            # Stop generation
            llm.lm_head.output.stop()
            break
            
        # Access next token
        with llm.lm_head.next().all():
            # Get token info
            logits = llm.lm_head.output
            probs = F.softmax(logits, dim=-1).save()
            current_token = logits.argmax(dim=-1)[0][-1].save()
            
            # Save values
            prob_values.append(probs)
            generated_tokens.append(current_token)
            
        # Increment counter
        token_index += 1

print(generated_tokens)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[tensor(314, device='mps:0')]


In [69]:
import torch
from transformers import AutoTokenizer
from transformer_lens import HookedTransformer
import torch.nn.functional as F
from scipy.stats import entropy

# Load model and tokenizer
model = HookedTransformer.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Function to calculate entropy
def calc_entropy(probs, axis=1):
    return entropy(probs.cpu().numpy(), axis=axis)

# Prompt and setup
prompt = "Once upon a time, "
n_new_tokens = 30
end_tokens = [13, 30, 0, 986]  # . ? ! ...

# Initialize storage
generated_tokens = []
entropy_values = []

# Tokenize the prompt
tokens = tokenizer.encode(prompt, return_tensors="pt")

# Generate tokens one by one
for i in range(n_new_tokens):
    # Get logits for next token
    with torch.no_grad():
        logits = model(tokens)[:, -1, :]
    
    # Convert to probabilities
    probs = F.softmax(logits, dim=-1)
    
    # Get the predicted token
    next_token = torch.argmax(logits, dim=-1).unsqueeze(0)
    
    # Calculate entropy
    token_entropy = calc_entropy(probs)
    
    # Add to lists
    generated_tokens.append(next_token.item())
    entropy_values.append(token_entropy)
    
    # Check for end token
    if next_token.item() in end_tokens:
        print(f"Found end token: {next_token.item()}")
        break
    
    # Add token to input for next iteration
    tokens = torch.cat([tokens, next_token], dim=1)

# Display results
print("Generated tokens:", generated_tokens)
print("Token text:", tokenizer.decode(generated_tokens))
print("Entropy values:", entropy_values)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loaded pretrained model gpt2 into HookedTransformer


RuntimeError: torch.cat(): all input tensors must be on the same device. Received cpu and mps:0